# The Currencies data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/Currencies.ipynb)

In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(RPostgres)
library(plutoR)
options("scipen"=999)
source("config.R")

#initialize
currencies <- Currencies()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



ERROR: Error in Currencies(): could not find function "Currencies"


### get traded futures pairs

In [2]:
currencies$NseFuturesTimeSeries() %>%
  group_by(SYMBOL) %>%
  summarize(START_DT = min(TIME_STAMP), END_DT = max(TIME_STAMP)) %>%
  arrange(START_DT) %>%
  print()

ERROR: Error in eval(lhs, parent, parent): object 'currencies' not found


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)